In [5]:
'''
This script hooks a layer from a loaded trained SchNet model for a forward pass (to output
values of intermediatte trained layers)
'''

import torch
import schnetpack as spk
import math


import numpy as np
import pandas as pd
from xlwt import Workbook



# Initiate  a) QM9 dataset and  
# b) SchNet model architechure as it is exactly saved checkpoint file that will be loaded later

# a) Load QM9 dataset
from schnetpack.datasets import QM9
qm9data = QM9('./qm9.db', download=True, remove_uncharacterized=True)

# split the data set according to saved split file (after training)
train, val, test = spk.data.train_test_split(qm9data, split_file='./trained_models/qm9i5e300/split.npz')

# load the test, train and validation dataset loaders
test_loader = spk.AtomsLoader(test, batch_size=100)
train_loader = spk.AtomsLoader(train, batch_size=100, shuffle=True)
val_loader = spk.AtomsLoader(val, batch_size=100)

# load some atom reference data that were used in inititial training conditions
atomrefs = qm9data.get_atomref(QM9.U0)
#print('U0 of hyrogen:', '{:.2f}'.format(atomrefs[QM9.U0][1][0]), 'eV')
#print('U0 of carbon:', '{:.2f}'.format(atomrefs[QM9.U0][6][0]), 'eV')
#print('U0 of oxygen:', '{:.2f}'.format(atomrefs[QM9.U0][8][0]), 'eV')


# get statistics on trained dataset
means, stddevs = train_loader.get_statistics(
    QM9.U0, divide_by_atoms=True, single_atom_ref=atomrefs)
#print('Mean atomization energy / atom:', means[QM9.U0])
#print('Std. dev. atomization energy / atom:', stddevs[QM9.U0])


# b) load SchNet representation model as it is exactly saved in checkpoint file
schnet = spk.representation.SchNet(
    n_atom_basis=30, n_filters=30, n_gaussians=20, n_interactions=5,
    cutoff=4., cutoff_network=spk.nn.cutoff.CosineCutoff
)
output_U0 = spk.atomistic.Atomwise(n_in=30, atomref=atomrefs[QM9.U0], property=QM9.U0,
                                   mean=means[QM9.U0], contributions=True, stddev=stddevs[QM9.U0])


# load atomistic model
model = spk.AtomisticModel(representation=schnet, output_modules=output_U0)


# Load saved checkpoint file
checkpoint_path = './trained_models/ckpt-trained.pth'
load_checkpoint = torch.load(checkpoint_path)


# load state dictionary from checkpoint
model.load_state_dict(load_checkpoint)

<All keys matched successfully>

In [6]:

#set up device that will be used in forward pass and atoms converter for input
device='cpu'

#load the test dataset
test_loader = spk.AtomsLoader(test, batch_size=100)

# load atoms converter 
converter = spk.data.AtomsConverter(device=device)

# load spk calculator
calculator = spk.interfaces.SpkCalculator(model=model, device=device, energy=QM9.U0)


In [7]:

# Set up excel sheets for for data extraction
wb1 = Workbook()
wb2 = Workbook()
wb3 = Workbook()

sheet1 = wb1.add_sheet('Sheet1')
sheet2 = wb2.add_sheet('Sheet1')
sheet3 = wb3.add_sheet('Sheet3')
index1=0
index2=0

# for a number of inputs, run a forward pass on interesting trained layers of SchNet model
for idx in range(500):
    
    #load atomistic data and properties from qm9data
    at, props = qm9data.get_properties(idx)
    
    # load x,y,z coordinates tensors
    x = props['_positions'][ :,0]
    y = props['_positions'][ :,1]
    z = props['_positions'][ :,2]
    x = x.numpy()
    y = y.numpy()
    z = z.numpy()
    
    # print atom coordinates for molecule identification
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            print('H',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 6:
            print('C',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 7:
            print('N',x[i],y[i],z[i])     
        if props['_atomic_numbers'][i] == 8:
            print('O',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 9:
            print('F',x[i],y[i],z[i]) 

    # set calculator to atomistic data
    at.set_calculator(calculator)
    # convert atomistic data to inputs for the model
    inputs = converter(at)
    
    
    # Hook the embedding layer for a forward pass
    
    
    #Instatiate the embedding_output
    embedding_output=None
    #Define a hooking function that will fetch the out_tensor 
    #from the layer of interest given an input tensor
    def embedding_hook(self, inp_tensor, out_tensor):
        # Self is included and refers to the model class
        # Global allows us to utilize embedding_output outside the current function scope
        global embedding_output
        #Update the embedding_output variable to be equal to our output tensor
        embedding_output=out_tensor 
    
    #Hook our function to the embedding layer during the forward pass
    model.representation.embedding.register_forward_hook(embedding_hook)

    #Check if None
    #Forward pass of the tensor inputs
    model(inputs)

    #embedding_output SHOULD be changed
    
    #We choose None to instatiate the variable originally
    interactions_output=None
    #Define a hooking function that will fetch the out_tensor 
    #from the layer of interest given an input tensor
    def interactions_hook(self, inp_tensor, out_tensor):
        # Self is included and refers to the model class
        # Global allows us to utilize embedding_output outside the current function scope
        global interactions_output
        #Update the embedding_output variable to be equal to our output tensor
        interactions_output=out_tensor 


    #Hook our function to the embedding layer during the forward pass
    model.representation.interactions[0].register_forward_hook(interactions_hook)

    #Check if None

    #Forward pass of the tensor inputs
    model(inputs)

    #embedding_output SHOULD be changed

    # Add the embedding tensor with the interaction tensor to get the final representation

    rep = interactions_output + embedding_output
    print(rep)
    print(rep[0][0][0])
    
    # convert rep tensor to a 2D array so that it can be easily copied to excel
    rep2D = np.zeros((len(z),30))
    for i in range(len(z)):
        for j in range(30):
            rep2D[i,j] = rep[0][i][j]
    
    
    # output data onto excel sheets

    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            place=i
            for j in range(30):
                sheet1.write(index1,j,rep2D[place,j])
            index1=index1+1
            break
    wb1.save('./data/rep-h.xls')
    read_file = pd.read_excel (r'./data/rep-h.xls')
    read_file.to_csv (r'./data/rep-h.csv', index = None, header=True)
    
    
    
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 8:
            place=i
            for j in range(30):
                sheet2.write(index2+1,j,rep2D[place,j])
        break
        
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 8:
            distance = np.zeros(len(z))
            if i != 0:
                distance[0] = math.sqrt((x[i]-x[0])**2+(y[i]-y[0])**2+(z[i]-z[0])**2)
            else:
                distance[0] = math.sqrt((x[i]-x[1])**2+(y[i]-y[1])**2+(z[i]-z[1])**2)
            lowest_distance = distance[0]
            for j in range(len(z)):
                distance[j] = math.sqrt((x[i]-x[j])**2+(y[i]-y[j])**2+(z[i]-z[j])**2)
                if distance[j] < lowest_distance and distance[j] != 0:
                    lowest_distance = distance[j]
                if props['_atomic_numbers'][j] == 1:
                    label = 1
                else:
                    label = 0                        
            sheet3.write(index2+1,0,label)                
            index2=index2+1
        break
            
    wb2.save('./data/rep-o.xls')
    read_file = pd.read_excel (r'./data/rep-o.xls')
    read_file.to_csv (r'./data/rep-o.csv', index = None, header=True)
    
    wb3.save('./data/label-o.xls')
    read_file = pd.read_excel (r'./data/label-o.xls')
    read_file.to_csv (r'./data/label-o.csv', index = None, header=True)
        

C -2.8340169e-06 2.3049886e-06 -1.4378233e-07
H 0.014845718 -1.0918331 -0.0060250196
H 1.0244261 0.3779494 -0.007724565
H -0.52811974 0.36172476 -0.88464487
H -0.5111183 0.3521308 0.89839613
tensor([[[-1.1397e+00, -1.1182e+00, -3.4455e-01, -1.6446e-01, -3.4134e-01,
          -1.5255e+00, -1.2848e+00,  8.1385e-01, -8.5612e-01, -1.5727e+00,
          -1.0041e-01, -2.8464e-04, -2.5764e-01, -1.1179e+00,  5.2213e-02,
           1.0528e+00, -6.1556e-02, -1.7145e+00,  1.4665e+00,  3.2707e-01,
           1.2560e+00, -9.2806e-01,  3.7177e-01,  2.2913e-01,  2.4961e-02,
          -5.7937e-01,  1.0286e+00,  2.1018e+00, -4.5133e-02, -4.1044e-01],
         [-1.4015e-01, -4.8519e-01,  5.1118e-01,  6.2755e-01,  7.2506e-01,
          -1.6231e+00,  1.4072e+00, -1.6591e-01, -7.5225e-01, -1.1311e+00,
          -1.6744e-01, -1.6576e-01, -2.5727e-01, -3.4328e-01,  1.0871e+00,
          -4.2440e-01,  1.2270e+00,  9.1800e-01,  3.7849e-01, -4.0430e-01,
          -1.6577e+00, -4.6592e-01, -9.5416e-01, -6.7781e-

C -0.010403622 0.7647314 0.004216852
C 0.010404125 -0.7647325 -0.004217135
H 1.0031731 1.1788927 -0.0032747546
H -0.53377575 1.1627601 -0.8713333
H -0.5169408 1.1533226 0.89380807
H 0.5337869 -1.1627584 0.871328
H -1.0031761 -1.1788844 0.0032925315
H 0.5169266 -1.1533209 -0.8938171
tensor([[[-1.2791, -0.8750, -0.2340, -0.3618,  0.1319, -1.2031, -1.2792,
           0.5850, -0.8724, -1.3589,  0.1321,  0.2663, -0.2877, -0.6085,
          -0.1131,  0.7017,  0.1230, -1.7820,  1.2700,  0.5423,  1.2375,
          -1.0230,  0.1890,  0.6882, -0.2674, -0.3839,  0.8798,  1.8546,
          -0.1995, -0.0402],
         [-1.2791, -0.8750, -0.2340, -0.3618,  0.1319, -1.2031, -1.2792,
           0.5850, -0.8724, -1.3589,  0.1321,  0.2663, -0.2877, -0.6085,
          -0.1131,  0.7017,  0.1230, -1.7820,  1.2700,  0.5423,  1.2375,
          -1.0230,  0.1890,  0.6882, -0.2674, -0.3839,  0.8798,  1.8546,
          -0.1995, -0.0402],
         [-0.2441, -0.4744,  0.6415,  0.7029,  0.6086, -1.7783,  1.4227,
  

tensor([[[-1.1723, -0.9507, -0.1934, -0.4231,  0.0496, -1.2053, -1.2699,
           0.5204, -0.8842, -1.3441,  0.0639,  0.2271, -0.2678, -0.5956,
           0.0139,  0.7668,  0.1674, -1.7524,  1.3221,  0.4981,  1.2429,
          -0.9967,  0.1761,  0.6614, -0.2831, -0.4696,  0.9105,  1.9185,
          -0.1382, -0.1044],
         [-1.2324, -0.7896, -0.1400, -0.7425,  0.4386, -0.7332, -1.1900,
           0.1636, -0.9789, -1.1506,  0.1551,  0.3090, -0.1671, -0.2242,
           0.1324,  0.4741,  0.3485, -1.6937,  1.2012,  0.6881,  1.0655,
          -1.1466,  0.0860,  1.0142, -0.5155, -0.4788,  0.8990,  1.6559,
          -0.1943,  0.0703],
         [-1.7641,  1.3583,  0.7985,  0.7046, -1.4162,  0.3922,  0.5724,
           0.7886, -0.1070, -0.7069,  1.9924, -0.6532,  1.4277,  1.2619,
          -0.5489, -2.6087, -0.2366,  2.0499,  1.7826, -1.3004,  1.7550,
          -0.9120, -1.6792,  0.2666, -0.0902, -0.1904, -1.0176,  1.2173,
          -0.3268, -0.0960],
         [-0.1684, -0.5296,  0.6435, 

tensor([[[-1.2036, -1.0072, -0.2037, -0.4611,  0.0726, -1.0259, -1.2482,
           0.4296, -0.9904, -1.3233, -0.0822,  0.1380, -0.1722, -0.6527,
           0.2590,  0.7616,  0.1950, -1.7026,  1.3170,  0.5710,  1.1665,
          -0.9683,  0.3230,  0.6291, -0.2933, -0.5187,  1.0526,  1.8563,
          -0.2104, -0.1485],
         [-1.4887, -0.5281, -0.0894, -0.6366,  0.9730, -0.7156, -1.1496,
           0.1807, -0.9960, -1.0168,  0.5017,  0.6782, -0.2862,  0.2533,
          -0.2565,  0.0897,  0.4941, -1.8846,  1.0291,  0.8252,  1.1772,
          -1.2136, -0.1387,  1.3560, -0.7347, -0.0223,  0.6022,  1.4886,
          -0.4156,  0.4363],
         [-1.2036, -1.0072, -0.2038, -0.4613,  0.0728, -1.0257, -1.2481,
           0.4294, -0.9904, -1.3233, -0.0822,  0.1380, -0.1721, -0.6526,
           0.2591,  0.7615,  0.1950, -1.7026,  1.3169,  0.5711,  1.1664,
          -0.9684,  0.3230,  0.6292, -0.2933, -0.5188,  1.0525,  1.8562,
          -0.2103, -0.1486],
         [-1.2036, -1.0072, -0.2038, 

C -0.027913883 2.0577686 0.011311069
C -0.008167857 0.6017288 0.003310354
C 0.008162743 -0.601729 -0.003307965
C 0.027917407 -2.0577683 -0.01131266
H 0.8545688 2.4668994 0.51599646
H -0.04221053 2.4631054 -1.0067108
H -0.9123655 2.442855 0.5312388
H -0.8448044 -2.4723904 0.5057136
H 0.022729926 -2.452156 -1.033713
H 0.92210054 -2.4483168 0.48746532
tensor([[[-1.1139, -0.8863, -0.1579, -0.6461,  0.1484, -0.8617, -1.1829,
           0.1540, -1.0973, -1.0638, -0.1083,  0.1299, -0.0613, -0.4189,
           0.5105,  0.7833,  0.2295, -1.6801,  1.2762,  0.5382,  1.1743,
          -0.9225,  0.0752,  0.6701, -0.2863, -0.7026,  1.0421,  1.8734,
          -0.1726, -0.1780],
         [-1.2457, -0.7143,  0.0420, -1.8063,  1.3499,  0.3103, -0.9195,
          -1.1075, -1.5503, -0.2406, -0.1501,  0.1718,  0.1926,  1.2207,
           1.3363, -0.0752,  1.0768, -1.8628,  1.1350,  0.7104,  0.7329,
          -1.3014, -0.5403,  1.4816, -0.8689, -0.9420,  1.2645,  1.0281,
          -0.2387,  0.3807],
       

...

O -1.8075937 -1.7729703 1.2510551
C -1.2546184 -1.3535764 0.27056012
C 0.047575057 -0.5769888 0.2828139
C -0.064275324 0.71922827 -0.55006117
C 1.2165762 1.4093511 -0.6741044
N 2.2374399 1.9426043 -0.7644319
H -1.6770153 -1.524778 -0.7479802
H 0.3307665 -0.366313 1.3161044
H 0.821107 -1.2180808 -0.16016172
H -0.43580514 0.49299 -1.5568166
H -0.7876826 1.4032321 -0.09257256
tensor([[[-1.7539,  1.3286,  0.7903, -0.2271, -0.8898,  0.9933,  0.7972,
          -0.1442, -0.4494, -0.1358,  1.6988, -0.8673,  1.7114,  2.2179,
           0.3706, -2.9683,  0.2806,  1.8546,  1.8390, -1.4229,  1.3966,
          -1.1462, -2.1220,  0.4711, -0.1181, -0.7229, -0.8059,  0.7748,
          -0.1503, -0.0508],
         [-1.1473, -0.9547,  0.0664, -1.4316,  0.9995, -0.1753, -0.9812,
          -0.5964, -1.4939, -0.8427, -0.0840,  0.3132,  0.0162,  0.6744,
           0.8946,  0.0759,  0.9933, -1.7681,  1.2225,  1.0017,  0.7668,
          -1.3392, -0.0774,  1.3787, -0.8297, -0.4605,  1.0107,  1.3783,
          -

O -1.7323531 -1.3226513 -0.6347938
C -1.7652564 -0.27921116 -0.023356896
N -0.69445306 0.28240335 0.6133051
C 0.6205886 -0.32529104 0.5509158
C 1.6213634 0.5667628 -0.04637317
N 2.408039 1.2858483 -0.49146166
H -2.6786585 0.33229935 0.09602957
H -0.80167645 1.1761252 1.0660269
H 0.9705146 -0.629295 1.5446616
H 0.5139139 -1.2284503 -0.05802385
tensor([[[-1.7495,  1.3910,  0.7827, -0.0715, -0.9290,  0.8708,  0.7671,
           0.0421, -0.3997, -0.2254,  1.8320, -0.8050,  1.6488,  2.1033,
           0.1041, -2.9322,  0.1523,  1.8439,  1.8421, -1.3944,  1.4583,
          -1.1147, -2.1183,  0.4613, -0.1138, -0.6266, -0.8867,  0.8369,
          -0.1239, -0.0032],
         [-1.0159, -1.0262,  0.0352, -1.4721,  0.9601, -0.1621, -0.9406,
          -0.6246, -1.4613, -0.8759, -0.1702,  0.2719,  0.1021,  0.6856,
           0.9745,  0.1638,  1.0362, -1.7407,  1.2518,  0.9155,  0.7012,
          -1.4170, -0.0791,  1.3230, -0.7729, -0.6342,  1.1030,  1.3851,
          -0.0817,  0.2351],
         [-1.

O -1.4279878 -2.4064448 0.7105827
C -1.3851653 -1.2110505 0.5867029
C -0.11960409 -0.38235575 0.6608028
C 0.119733 0.38224143 -0.6605726
C 1.3852009 1.2111199 -0.5866055
O 1.4278392 2.4064798 -0.71087295
H -2.308928 -0.61837584 0.3918324
H -0.23435673 0.34238595 1.4783287
H 0.7171377 -1.045512 0.89807016
H 0.23454013 -0.3425042 -1.4780903
H -0.7170545 1.0453236 -0.8978875
H 2.3090618 0.618663 -0.39154834
tensor([[[-1.7498,  1.3291,  0.7884, -0.2187, -0.8960,  0.9875,  0.7950,
          -0.1337, -0.4495, -0.1389,  1.7040, -0.8680,  1.7111,  2.2116,
           0.3628, -2.9579,  0.2749,  1.8540,  1.8342, -1.4228,  1.4034,
          -1.1427, -2.1234,  0.4663, -0.1165, -0.7239, -0.8065,  0.7800,
          -0.1472, -0.0481],
         [-1.1471, -0.9542,  0.0658, -1.4310,  0.9978, -0.1710, -0.9825,
          -0.5984, -1.4914, -0.8414, -0.0848,  0.3115,  0.0173,  0.6714,
           0.8976,  0.0770,  0.9924, -1.7645,  1.2221,  1.0006,  0.7654,
          -1.3388, -0.0746,  1.3791, -0.8292, -0.463

O -1.3210258 -2.3663359 0.90262485
C -1.0452055 -1.3478597 0.33101007
C 0.0975221 -0.43874788 0.7691451
O 0.032910068 0.7962966 0.068965
C 1.0811877 1.1495063 -0.71066606
O 1.1013799 2.161094 -1.3379043
H -1.5942149 -0.9859108 -0.5656278
H 0.020035347 -0.24828148 1.8440573
H 1.0499575 -0.95596194 0.5849793
H 1.897306 0.4004097 -0.69028175
tensor([[[-1.7557,  1.3191,  0.8091, -0.2577, -0.8765,  1.0070,  0.8072,
          -0.1743, -0.4623, -0.1381,  1.6801, -0.8583,  1.7017,  2.2436,
           0.3904, -2.9972,  0.3135,  1.8615,  1.8576, -1.4093,  1.3765,
          -1.1665, -2.1081,  0.4949, -0.1338, -0.6967, -0.8098,  0.7615,
          -0.1571, -0.0438],
         [-1.0773, -0.9645,  0.0962, -1.4566,  0.9351, -0.2677, -0.9741,
          -0.5806, -1.4803, -0.8189, -0.0763,  0.3461, -0.0356,  0.7215,
           0.8545,  0.1028,  1.0109, -1.7840,  1.2623,  0.9493,  0.8196,
          -1.3208, -0.1575,  1.3746, -0.8282, -0.4666,  0.9476,  1.4360,
          -0.1439,  0.2935],
         [-1.0502

C 0.56264234 3.1150515 0.32450163
C 0.55451787 1.660008 0.2819443
C 0.5669935 0.4566715 0.24873629
C 0.5173006 -0.99689484 0.18802856
C -0.88899094 -1.5168307 -0.118331626
O -1.1115417 -2.4657602 -0.81938523
H 1.5171568 3.5148578 -0.03476898
H -0.22978915 3.5354137 -0.30423725
H 0.41021848 3.4854274 1.3440732
H 1.2035147 -1.4110043 -0.55807555
H 0.8058151 -1.4290714 1.1591067
H -1.7033849 -0.9458835 0.37864414
tensor([[[-1.1136, -0.8864, -0.1576, -0.6450,  0.1472, -0.8624, -1.1832,
           0.1549, -1.0972, -1.0640, -0.1084,  0.1296, -0.0616, -0.4199,
           0.5101,  0.7841,  0.2289, -1.6797,  1.2763,  0.5377,  1.1751,
          -0.9218,  0.0753,  0.6691, -0.2861, -0.7025,  1.0423,  1.8741,
          -0.1729, -0.1785],
         [-1.2497, -0.7119,  0.0414, -1.8507,  1.3579,  0.3354, -0.9170,
          -1.1520, -1.5597, -0.2057, -0.1607,  0.1571,  0.1968,  1.2588,
           1.3779, -0.0947,  1.0948, -1.8645,  1.1382,  0.6932,  0.7222,
          -1.3093, -0.5683,  1.4799, -0.8608, 